In [1]:
import sympy as sp
from sympy.parsing.latex import parse_latex
from IPython.display import Math, display
import re

def simplify_and_display_latex(latex_str):
    try:
        
        # --- 2. Parse ---
        expr = parse_latex(latex_str)
        
        # --- 3. Intelligent Substitution ---
        sym_replacements = {}
        func_replacements = {}
        
        # A. Symbol Replacements
        for sym in expr.atoms(sp.Symbol):
            name = sym.name
            if name == 'pi': sym_replacements[sym] = sp.pi
            elif name == 'e': sym_replacements[sym] = sp.exp(1)
            elif name == 'i': sym_replacements[sym] = sp.I
            elif name == 'infty': sym_replacements[sym] = sp.oo
            # Variables
            elif name in ['x', 't', 'omega', 'ω', 's', 'y', 'a', 'b']:
                sym_replacements[sym] = sp.Symbol(name, real=True)
            # Loop Variables (Integer + Positive helps Sum convergence)
            elif name in ['n', 'k']:
                sym_replacements[sym] = sp.Symbol(name, integer=True, positive=True)
            elif sym not in sym_replacements:
                sym_replacements[sym] = sp.Symbol(name, real=True)

        # B. Function Replacements
        for func in expr.atoms(sp.Function):
            func_type = type(func)
            name = func_type.__name__
            if name == 'zeta': func_replacements[func_type] = sp.zeta
            elif name == 'Gamma': func_replacements[func_type] = sp.gamma
            elif name == 'delta': func_replacements[func_type] = sp.DiracDelta

        # C. Apply Substitutions
        if sym_replacements: expr = expr.subs(sym_replacements)
        if func_replacements: expr = expr.subs(func_replacements)

        # --- 4. Strategy Pipeline ---
        
        candidates = {}
        
        # Pre-calculate common forms
        expr_expanded = sp.expand(expr)
        
        # Strategy 1: Standard (Doit -> Simplify)
        # Good for general integrals/sums
        base_doit = expr.doit(deep=True)
        candidates['standard'] = sp.simplify(base_doit)
        
        # Strategy 2: Expander (Doit -> Expand -> Simplify)
        # Good for derivatives that produce cancelling polynomials
        candidates['post_expand'] = sp.simplify(sp.expand(base_doit))
        
        # Strategy 3: Expand-First (Expand -> Doit -> Simplify)
        # Good when the unexpanded input confuses the derivative engine
        candidates['pre_expand'] = sp.simplify(expr_expanded.doit(deep=True))
        
        # Strategy 4: Unified (Trig <-> Exp)
        try:
            candidates['unified'] = sp.simplify(base_doit.rewrite(sp.exp))
        except:
            pass

        # --- 5. Selection Logic ---
        final_result = candidates['standard']
        found_zero = False
        
        # Check for exact symbolic zero
        for key, val in candidates.items():
            if val == 0:
                final_result = 0
                found_zero = True
                break
        
        # --- 6. Robust Numerical Safety Net ---
        if not found_zero and not isinstance(final_result, type):
            # We try graduated precision levels. 
            # High precision is good for accuracy, Low precision is good for singularities.
            precision_levels = [15, 10, 5] 
            
            for prec in precision_levels:
                if found_zero: break
                try:
                    # evalf can handle Sums/Integrals that symbolic engine failed on
                    numeric_val = expr.evalf(n=prec)
                    
                    if hasattr(numeric_val, 'is_Number') and numeric_val.is_Number:
                        # Threshold scales with precision roughly
                        threshold = 10**(-prec + 2) 
                        if abs(numeric_val) < threshold:
                            final_result = 0
                            found_zero = True
                except:
                    continue

        # --- 7. Formatting ---
        if not found_zero:
            res_std = candidates['standard']
            res_exp = sp.expand(res_std)
            if res_std != res_exp and sp.count_ops(res_exp) < 60:
                final_result = res_exp
            else:
                final_result = res_std

        print("Input:")
        display(Math(latex_str))
        print("Output:")
        display(Math(sp.latex(final_result)))
        
    except Exception as error:
        print(f"An error occurred: {error}")

# --- Interactive Usage ---
print("\n--- LaTeX Simplifier ---")
print("Type your LaTeX string below (or press Enter to quit).\n")
while True:
    latex_input = input("\nLaTeX >> ").strip()
    if not latex_input: 
        print("Exiting...")
        break
    simplify_and_display_latex(latex_input)


--- LaTeX Simplifier ---
Type your LaTeX string below (or press Enter to quit).

Input:


<IPython.core.display.Math object>

Output:


<IPython.core.display.Math object>

Input:


...

Input:


<IPython.core.display.Math object>

Output:


<IPython.core.display.Math object>

Exiting...
